# Extremes using ORAS5 Reanalysis

## Import libraries

In [ ]:
from collections import defaultdict

import pandas as pd
import xarray as xr
from c3s_eqc_automatic_quality_control import diagnostics, download, utils

## Define parameters

In [ ]:
# Time
start = "1979-01"
stop = "2016-01"

# Space
lat_slice = slice(40, 50)
lon_slice = slice(-150, -130)

# Variables
variables = [
    "meridional_wind_stress",
    "mixed_layer_depth_0_03",
    "net_downward_heat_flux",
    "zonal_wind_stress",
]

## Define request

In [ ]:
# Define request
collection_id = "reanalysis-oras5"

requests = defaultdict(list)
for variable in variables:
    for date in pd.date_range(start, stop, freq="1MS"):
        requests[variable].append(
            {
                "vertical_resolution": "single_level",
                "variable": variable,
                "product_type": "operational" if date.year > 2014 else "consolidated",
                "year": date.strftime("%Y"),
                "month": date.strftime("%m"),
            }
        )

grid_request = (
    collection_id,
    {
        "vertical_resolution": "single_level",
        "variable": "net_downward_heat_flux",
        "product_type": "consolidated",
        "year": "1979",
        "month": "01",
    },
)

## Functions to cache

In [ ]:
def regionalise_and_interpolate(
    ds, lat_slice, lon_slice, grid_request, **regrid_kwargs
):
    ds_grid = download.download_and_transform(
        *grid_request,
        transform_func=utils.regionalise,
        transform_func_kwargs={"lon_slice": lon_slice, "lat_slice": lat_slice},
        quiet=True,
    )
    return diagnostics.regrid(ds, ds_grid, **regrid_kwargs)

## Download and transform

In [ ]:
datasets = []
for variable, request in requests.items():
    print(f"{variable=}")
    transform_func_kwargs = {"lon_slice": lon_slice, "lat_slice": lat_slice}
    if "wind_stress" in variable:
        transform_func = regionalise_and_interpolate
        transform_func_kwargs.update(
            {"grid_request": grid_request, "method": "nearest_s2d"}
        )
    else:
        transform_func = utils.regionalise
    rolling = {}
    ds = download.download_and_transform(
        collection_id,
        request,
        transform_func=transform_func,
        transform_func_kwargs=transform_func_kwargs,
    )
    datasets.append(ds)
ds = xr.merge(datasets, compat="override")